In [1]:
import sympy as sp
import minterpy as mp
import numpy as np
from minterpy.pointcloud_utils import *

from mpl_toolkits import mplot3d
%matplotlib inline

from mpl_toolkits.mplot3d import axes3d

import torch
import torchvision
from torchvision import transforms, datasets

import random
import numpy as np
import matplotlib.pyplot as plt

import os
from operator import itemgetter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import ot

from sklearn.neighbors import NearestNeighbors

import ripser
import persim
from persim import plot_diagrams

from operator import itemgetter


#do pip installs as follows to use vedo for plotting point clouds
#pip install vedo
#pip install ipyvtklink

import numpy as np
from vedo import *

import matplotlib.pyplot
from mpl_toolkits.mplot3d import Axes3D

In [2]:
def give_centeroid(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_z = np.sum(arr[:, 2])
    return np.array([[sum_x/length, sum_y/length, sum_z/length]])

In [129]:
def give_next_neighbours_barycenter_indices(batch_x, input_barycenter, remaining_indices, no_of_barycenetrs_required):
    

  #num_neighbours = int(batch_x.shape[0] / no_of_barycenetrs_required)
    
  wasserDistance = []
  distance_cum_index = np.array([])

  for j in remaining_indices:
    
    wassDistance = dist = np.linalg.norm(batch_x[j]-input_barycenter)

    
    distance_cum_index = np.concatenate((distance_cum_index, np.array([wassDistance, j])), axis = 0)
   
  distance_cum_index = distance_cum_index.reshape(int(distance_cum_index.shape[0]/2), 2)
  distance_cum_index = sorted(distance_cum_index, key=itemgetter(0))
  
  distance_cum_index = np.array(distance_cum_index)
  only_distances = distance_cum_index[:,0]
  print(only_distances)
  where_is_it = np.where( only_distances < 0.5 ) 
  print('where_is_it', where_is_it)  
  print('where_is_it[0][-1]', where_is_it[0][-1])
  num_neighbours = where_is_it[0][-1] +1
  
  remaining_indices = distance_cum_index[:,1]

  remaining_indices = remaining_indices.astype(int)
  
    
  A = np.array([])
  for i in range(num_neighbours):
    if(i >= distance_cum_index[:,1].shape[0]):
        break
    A = np.concatenate((A, batch_x[int(distance_cum_index[:,1][i])]), axis = 0 )
  
  A = A.reshape(int(A.shape[0]/3) , 3)
  
  #print("The shape of A is ")
  #print(A.shape)
  next_barycenter = give_centeroid(A) 
  
  next_barycenter = np.array(next_barycenter)
  #print(next_barycenter.shape)
  next_barycenter = next_barycenter.reshape(next_barycenter.shape[0]* next_barycenter.shape[1])
  print(next_barycenter)  
  return A, next_barycenter, remaining_indices,num_neighbours


In [130]:
def get_convergent_barycenters(point_cloud, no_of_barycenetrs_required, initial_pt):   
    
    #no_neighbours = int(point_cloud.shape[0] / no_of_barycenetrs_required)
    #initiating no of neighbours
    no_neighbours = 5
    
    #num_neighbours = int(batch_x.shape[0] / no_of_barycenetrs_required)

    
    bary = initial_pt
    rem_indices = np.array(range(0,point_cloud.shape[0]))
    #print("Size of batch : ", point_cloud.shape[0])
    sampled_barycenters = np.array([])
    sampled_barycenters = torch.tensor(sampled_barycenters)
    covered_indices = np.array([])

    for i in range(int(point_cloud.shape[0])):

        if(len(rem_indices) < 2):
            #print("END")
            break

        #print("Iteration number : ", i+1)
        #print("Input barycenter : ")


        old_bary = bary


        print('rem_indices before', rem_indices)
        neighbours, bary, rem_indices, no_neighbours = give_next_neighbours_barycenter_indices(point_cloud, bary, rem_indices, no_neighbours)
        print('no_neighbours', no_neighbours)

        wassDistance = np.linalg.norm(old_bary - bary)  

        covered_indices = np.concatenate((covered_indices, rem_indices[:4] ) ,axis = 0)


        print("Tracking distance between new barycenter and previous barycenter : ",wassDistance )
        if(wassDistance < 0.000001):

            unique_covered_indices = np.unique(covered_indices, axis=0)
            sampled_barycenters = torch.cat((sampled_barycenters, torch.tensor(bary)), 0)

            s1 = set(rem_indices)
            s2 = set(unique_covered_indices)
            rem_set = s1 - s2
            rem_inds = list(rem_set)
            #rem_indices = rem_set
            rem_indices = rem_indices[no_neighbours:]

            #print("Sampled barycenters are")
            #print(sampled_barycenters)
            #no_neighbours = 5
            print('rem_indices',rem_indices)
            print("len(rem_indices)",len(rem_indices))
            if(len(rem_indices) ==0):
                break
            bary = point_cloud[rem_indices[0]]
    sampled_barycenters = sampled_barycenters.reshape(int((sampled_barycenters.shape[0]/3)),3)
    
    return sampled_barycenters

In [95]:
def _compute_distance_matrix(x, p=2):
    x_flat = x.view(x.size(0), -1)

    distances = torch.norm(x_flat[:, None] - x_flat, dim=2, p=p)

    return distances

In [35]:
def get_persistence_diagram(point_cloud, maximum_dim):

    point_cloud = torch.tensor(point_cloud)

    dist_matrix = _compute_distance_matrix(point_cloud, p=2)
    diagrams = ripser.ripser(dist_matrix.cpu().detach().numpy(), distance_matrix=True, maxdim=maximum_dim)['dgms']
    return diagrams, plot_diagrams(diagrams, show=True)


# Torus in 3D

In [7]:
x, y, z = sp.symbols('x y z')


In [8]:
## Torus
R = 1.5
r = 0.7
expr_torus = (x**2 + y**2 + z**2 + R**2 - r**2)**2 - 4*R**2*(x**2 + y**2)


In [9]:
expr_torus

-9.0*x**2 - 9.0*y**2 + 3.0976*(0.568181818181818*x**2 + 0.568181818181818*y**2 + 0.568181818181818*z**2 + 1)**2

In [10]:
poly_torus = sp.Poly(expr_torus, x, y, z)

In [11]:
sp.expand(expr_torus)

1.0*x**4 + 2.0*x**2*y**2 + 2.0*x**2*z**2 - 5.48*x**2 + 1.0*y**4 + 2.0*y**2*z**2 - 5.48*y**2 + 1.0*z**4 + 3.52*z**2 + 3.0976

In [12]:
#Convert sympy polynomial to minterpy polynomial

newt_poly_torus = sympy_to_mp(poly_torus, mp.NewtonPolynomial)

# Trials on Torus : 

In [13]:
#Sample points on the zero isocontour
def points_on_torus(num_points):
    
    point_data_torus = sample_points_on_poly(num_points,        # Number of points to be sampled
                                       newt_poly_torus,  # Polynomial in Newton basis
                                       bounds=3.0, # Boundary of the Cubic domain to be sampled
                                       tol=1e-15)  # Tolerance in solution
    
    return point_data_torus

# 1. 1000 barycenters from 10000 points

In [118]:
#Sample points on the zero isocontour

point_data_torus = sample_points_on_poly(1000,        # Number of points to be sampled
                                   newt_poly_torus,  # Polynomial in Newton basis
                                   bounds=3.0, # Boundary of the Cubic domain to be sampled
                                   tol=1e-15)  # Tolerance in solution

In [119]:
point_data_torus.shape

(1000, 3)

In [120]:
x_t = point_data_torus[:,0]
y_t = point_data_torus[:,1]
z_t = point_data_torus[:,2]


plot_points = [x_t,y_t,z_t]
Points(plot_points).show(axes=1)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

# demo of barycenter convergence on a torus


In [131]:
torus_bary = get_convergent_barycenters(point_data_torus, 10, point_data_torus[0])

rem_indices before [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241

where_is_it (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23]),)
where_is_it[0][-1] 23
[-0.95039862 -0.46316024  0.51258761]
no_neighbours 24
Tracking distance between new barycenter and previous barycenter :  0.07847849766582558
rem_indices before [552 450   6 940 366 925 677 962 722  26 743 950 510 781 138 957 639 634
  98 385 361 580 107  31  67  50 266 990  19 519 240 151 699 503 869 614
 488 436 569 900 269 913  17 303 822 770 782 145 558 372 847 912 392 577
 885 903 323  90 391 665 422 720 646  43 121 100 278  34 465 585 406 642
 523 345 428 486 884 466 621 901 850 897 207 568 849 563 368  41 103  96
  94 618 500 147  75 786 244 831 827 952 874  78  14 186 228 196 324 162
 682 236 243 320 509 723 218  77 524 520 117 932 277 832 919 551 464 272
 845 658 695 284 448  42 685 199  74 727 974 467 892 991 542 322 273 859
 872  89 989 576 895 894 797 477 418 106 752 816 355 327 499  63 128   4
 909 623 126 449 360 571 603 941 

[0.05663685 0.17897675 0.22477155 0.22631821 0.25933943 0.39340288
 0.39902236 0.41389313 0.4376284  0.43779739 0.44645574 0.46452362
 0.46622471 0.46800674 0.51132002 0.55305131 0.554108   0.56983055
 0.57467984 0.63192612 0.65099192 0.65920614 0.67008131 0.71763359
 0.7313686  0.74788865 0.75682478 0.77601692 0.80574774 0.82283835
 0.85652141 0.85713976 0.85906561 0.86307578 0.86566037 0.86651892
 0.86732932 0.87743273 0.87872441 0.88846832 0.92684907 0.92899361
 0.93131865 0.93710816 0.93957359 0.93976841 0.94217062 0.95879569
 0.96869432 0.9703324  0.97991876 0.98051956 1.00391412 1.00847872
 1.00863814 1.02847437 1.03502421 1.03553297 1.03963051 1.04006185
 1.04318517 1.04845214 1.05448145 1.06089783 1.06249637 1.08007849
 1.0811919  1.08208593 1.08465578 1.09194398 1.09991039 1.10167978
 1.10315884 1.10387951 1.11338371 1.11715307 1.12498317 1.12522848
 1.12986929 1.13141448 1.14143018 1.1420791  1.14650885 1.16323339
 1.16782113 1.16904241 1.16960861 1.17032828 1.17072585 1.1722

[0.14013703 0.15288118 0.16981003 0.19781563 0.2052826  0.2392748
 0.24112776 0.27048495 0.28554702 0.30067238 0.33864044 0.34381324
 0.35629925 0.3567652  0.39104801 0.39323542 0.4083252  0.40986608
 0.41031016 0.41226874 0.41302947 0.42483481 0.44394216 0.44482204
 0.46157752 0.48412136 0.48942843 0.50466209 0.5167772  0.53064993
 0.53538946 0.55901824 0.57212175 0.57239619 0.57495691 0.5984036
 0.61265088 0.62247142 0.63578224 0.64150654 0.65413016 0.65705417
 0.65977298 0.67124411 0.67606986 0.69166964 0.69543362 0.69587285
 0.71103199 0.717458   0.73298269 0.73917311 0.75074958 0.76121043
 0.77319956 0.77823692 0.79233267 0.80161373 0.80978796 0.83322718
 0.83920417 0.86779374 0.87194268 0.87333246 0.87658595 0.88095017
 0.88178364 0.88504431 0.89116029 0.89721451 0.90543469 0.91473899
 0.91894905 0.92881064 0.93524341 0.93644037 0.94850564 0.94916475
 0.95148148 0.96667216 0.973119   0.98338159 0.99663357 0.99948591
 1.00693679 1.02818818 1.0302284  1.03130419 1.0390769  1.040298

[0.0933789  0.10939328 0.11784193 0.141463   0.20902162 0.21301505
 0.27787232 0.30839207 0.31929853 0.34006684 0.36907691 0.44337984
 0.44516372 0.45904079 0.4648277  0.47605934 0.48240478 0.49671988
 0.52354527 0.54935234 0.57776871 0.59215977 0.59704749 0.6002329
 0.6218903  0.62587338 0.63832566 0.64422196 0.64484829 0.66463722
 0.67942348 0.68998535 0.69025405 0.6911956  0.70150073 0.70245165
 0.70443293 0.71961122 0.72053297 0.74101732 0.74911921 0.75629695
 0.76756997 0.7832123  0.78800428 0.7905214  0.79630912 0.79661274
 0.81136348 0.81381598 0.84088127 0.88467063 0.92170595 0.95659793
 0.98685576 0.98916596 0.9942896  1.00334821 1.0037082  1.0082761
 1.02466428 1.03526606 1.0384816  1.04384828 1.055104   1.05915458
 1.05950354 1.08370127 1.086161   1.08755303 1.09038786 1.09336456
 1.10271949 1.10329936 1.11621615 1.12924229 1.13175529 1.13438558
 1.13893982 1.14444185 1.14900155 1.15000645 1.15340967 1.1620447
 1.16736799 1.17069553 1.17572492 1.17630615 1.17762263 1.1794266

...

[0.05805138 0.08774738 0.16180908 0.17910862 0.1971208  0.21318142
 0.22883616 0.24859078 0.29805794 0.30847174 0.35300191 0.35673616
 0.39762769 0.48844251 0.50806823 0.5841051  0.58610761 0.59869833
 0.61384443 0.61823602 0.62479033 0.6400685  0.69356606 0.70736426
 0.72055715 0.74733011 0.76974392 0.78304293 0.80472899 0.8789498
 0.89413403 0.8965199  0.89940014 0.90448132 0.92653969 0.96129234
 0.96306057 0.97111563 0.99073683 0.99360652 0.99654848 1.0034534
 1.00828697 1.01877399 1.0459534  1.04612142 1.05531358 1.07382225
 1.08167519 1.08170279 1.0856987  1.09201101 1.10569289 1.10675798
 1.12271218 1.12272385 1.12425064 1.14172133 1.1443705  1.15192397
 1.15959309 1.17055666 1.20039415 1.21073162 1.21099959 1.21847804
 1.22401709 1.24663423 1.25049899 1.25086318 1.25427    1.25564971
 1.2620388  1.26683788 1.26825809 1.27613518 1.27706667 1.27898624
 1.28595203 1.2916258  1.29479471 1.29498529 1.29866148 1.30106667
 1.30192066 1.30609459 1.30899394 1.31457071 1.3153286  1.323234

[0.         0.1134958  0.16412898 0.25347592 0.30564964 0.32305808
 0.33267211 0.36623893 0.40154325 0.44567832 0.44609299 0.44858637
 0.476553   0.50545891 0.53929083 0.54647651 0.5562664  0.56444931
 0.5812375  0.58611916 0.62106665 0.62874901 0.65191045 0.65680233
 0.65970342 0.66817557 0.67223052 0.67744796 0.69832654 0.70977475
 0.71918641 0.77139935 0.78375616 0.78746522 0.80408024 0.80616751
 0.80652787 0.80697639 0.82233363 0.82460972 0.84489897 0.85101594
 0.85619623 0.8569334  0.87347428 0.88935929 0.90373246 0.9091236
 0.92451841 0.9306068  0.93556806 0.9570483  0.96733677 0.97949072
 0.9829695  0.99689046 1.00662323 1.01440062 1.01817917 1.02421502
 1.02526104 1.0262744  1.03513285 1.06593827 1.07304746 1.07928167
 1.08231997 1.08899086 1.08907175 1.10175512 1.11259265 1.11514852
 1.11660862 1.11824319 1.12436568 1.12441354 1.12594879 1.13512594
 1.13671454 1.13758581 1.14292157 1.1435089  1.15140294 1.15507823
 1.1620273  1.16355352 1.16486595 1.16830821 1.16844142 1.17358

[0.12958843 0.13122912 0.13359982 0.15879452 0.16326634 0.22828562
 0.23820599 0.25166868 0.2577858  0.26092824 0.26099227 0.26717373
 0.27559998 0.27762672 0.28758397 0.28992443 0.29486631 0.31293015
 0.3374975  0.35958071 0.38262318 0.40799925 0.42894259 0.44569398
 0.44741401 0.45319703 0.46278478 0.47361984 0.51143996 0.53818149
 0.53916843 0.55253258 0.58716741 0.64604661 0.65567329 0.65592269
 0.72243196 0.74535132 0.78236066 0.78353945 0.78755002 0.80259505
 0.81249311 0.83641429 0.90286212 0.91035456 0.94960614 0.97987967
 0.98990982 1.03343951 1.05477174 1.08219314 1.09488134 1.09511551
 1.09829098 1.12140277 1.13021733 1.16583327 1.16877679 1.20117657
 1.24227581 1.2623927  1.28448189 1.29269775 1.30598112 1.31462115
 1.31994855 1.32315412 1.36384224 1.36492937 1.37222547 1.38026115
 1.38559379 1.39351075 1.40219301 1.40265425 1.40917573 1.41862905
 1.4215357  1.43757012 1.47144996 1.48692414 1.51625138 1.53075318
 1.53212332 1.5679189  1.57554548 1.57858354 1.57956819 1.6128

[0.03139082 0.04835839 0.09587419 0.12475749 0.1473901  0.54941806
 0.60548011 0.73082469 0.7328893  0.81787557 0.86785757 0.87439256
 0.95647627 0.96090182 0.97711403 0.98231395 0.98833019 1.08904498
 1.09416517 1.11109967 1.20428044 1.20841537 1.28535141 1.29626634
 1.3034143  1.31646114 1.36238401 1.37802053 1.38343872 1.38915596
 1.39783059 1.41474122 1.41687408 1.41713989 1.42503434 1.44789395
 1.47951337 1.48001158 1.48804352 1.50069221 1.50664758 1.52781186
 1.53464225 1.53621743 1.54987518 1.55157473 1.551888   1.56066523
 1.58204643 1.58879285 1.59648849 1.62650541 1.64822749 1.64997108
 1.65273286 1.65327551 1.65380588 1.70174084 1.7173366  1.72086416
 1.72612549 1.74795761 1.7541436  1.75539104 1.77307112 1.77770899
 1.7893364  1.79078339 1.81650891 1.82314159 1.83834936 1.84104961
 1.85265031 1.85787401 1.87265021 1.87418805 1.87530546 1.88430019
 1.88986385 1.89322396 1.89413894 1.90283563 1.91131389 1.91562182
 1.91916847 1.92545207 1.9529051  1.96288114 1.96974252 1.9743

where_is_it[0][-1] 24
[-0.14530419  1.5828338   0.64887112]
no_neighbours 25
Tracking distance between new barycenter and previous barycenter :  0.05480074753677068
rem_indices before [291 394 455 177 596 517 539 246 109 111  82 454 615 460 506  27 628 328
 549 573 259  49 696 484 754 313 815 981 353 217 690 609 408 985 504 626
 784 999 144 949 821  97 257 232 996 877 302 922 625  12 164 772 512 793
 620 471 415 521 249 687 910 921 956 965 704 290 856 195 379 118 668 214
 548 522 809 494 656 490   9  13 735 167 691 205 924 335 248 916 265 648
 970  50 112 953  71 386 447 382 744 794 745  25 273 912 537 991 387 229
 563 894 277 480 100 579 283 823 866 310 207 652 344 897 850 399 863 568
  46 237 800  77 243 919 411  74 509 750 756 752  73 126 285 459 836   7
 533 133 755]
[0.04987002 0.16520838 0.22789195 0.25479747 0.25604726 0.25793417
 0.26133921 0.27240647 0.30836731 0.31910915 0.32760339 0.34547241
 0.35115962 0.37360335 0.37590495 0.37868973 0.39014667 0.40489395
 0.43000262 0.432

In [132]:
torus_bary.shape

torch.Size([76, 3])

In [133]:
# plotting the sampled barycenters

x_stb = torus_bary[:,0]#.cpu().detach().numpy()

y_stb = torus_bary[:,1]#.cpu().detach().numpy()

z_stb = torus_bary[:,2]#.cpu().detach().numpy()



plot_points = [x_stb,y_stb,z_stb]
Points(plot_points).show(axes=1)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)